In [ ]:
# !ls '/content/drive/My Drive/Research/SAnet/data_files/session3'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import matplotlib.pyplot as plt
import torch
# Neural networks can be constructed using the torch.nn package.
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Subset
from torch.utils.data.sampler import SequentialSampler
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms

In [ ]:
%cd '/content/drive/My Drive/Research/SAnet/yolov5/yolov5'
!pwd

/content/drive/My Drive/Research/SAnet/yolov5/yolov5
/content/drive/My Drive/Research/SAnet/yolov5/yolov5


In [ ]:
# run this

ANNOS_PATH = '/content/drive/My Drive/Research/SAnet/data_files/annos/'
DATA_PATH = '/content/drive/My Drive/Research/SAnet/data_files/'

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:

def read_anno_file(file_path):
    session_annos = pd.read_csv(file_path)[['Image','O_loc','EE_loc']]
    session_annos['session'] = int( (file_path).split('_annos.csv')[0][-1] )
    return session_annos

## Read annotation files
# session1_annos = read_anno_file(ANNOS_PATH+'session1_annos.csv')
# session2_annos = read_anno_file(ANNOS_PATH+'session2_annos.csv')
session3_annos = read_anno_file(ANNOS_PATH+'session3_annos.csv')
session4_annos = read_anno_file(ANNOS_PATH+'session4_annos.csv')
session5_annos = read_anno_file(ANNOS_PATH+'session5_annos.csv')
session6_annos = read_anno_file(ANNOS_PATH+'session6_annos.csv')
session7_annos = read_anno_file(ANNOS_PATH+'session7_annos.csv')
# session4_annos = read_anno_file(ANNOS_PATH+'session4_annos.csv')

## Concatenate annotaion frames
data = pd.concat([
                  # session1_annos, 
                  # session2_annos, 
                  session3_annos, 
                  session4_annos, 
                  session5_annos, 
                  session6_annos, 
                  session7_annos]).reset_index(drop=True)

data.loc[(data.O_loc == 0) & (data.EE_loc == 2), 'O_loc'] = 2
#data.groupby(['O_loc','EE_loc']).size().reset_index(name='counts')

print('annotations',data.head())



## read all images to images_lib
images_lib = {}
for index in range(data.shape[0]):
    print(index)
    session_name = 'session'+str(data.loc[index, 'session'])
    img_name = session_name+'-'+str(data.loc[index, 'Image'])+'.jpg'
    img_name = os.path.join(DATA_PATH+session_name, img_name)
    image = Image.open(img_name)
    
    #image = image.convert('RGB')
    image = image.resize((640,480))
    label = torch.tensor(data.loc[index, ['EE_loc', 'O_loc']])
    if transform is not None:
        image = transform(image)
        images_lib[img_name] = image

annotations    Image  O_loc  EE_loc  session
0      1      0       0        3
1      2      0       0        3
2      3      3       3        3
3      4      3       3        3
4      5      3       3        3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224


In [ ]:


## Shuffling and splitting dataset into train val test dataset. Split = 0.7:0.15:0.15
batch_size = 32
shuffle_dataset = False
random_seed= 42
dataset_size = len(data) 
indices = list(range(dataset_size))
split1 = int(np.floor(0.7 * dataset_size))
split2 = int(np.floor(0.15 * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
# train_indices, val_indices ,test_indices  = indices[:split1], indices[split1:split1+split2], indices[split1+split2:]
# train_indices, val_indices ,test_indices, test1_indices  = indices[:750], indices[750:982], indices[982:1264], indices[1264:]
# train_indices, val_indices ,test_indices, test1_indices  = indices[:1315], indices[1315:1315+507], indices[1315+507:], indices[1315+507:]
# train_indices, val_indices ,test_indices, test1_indices  = indices[:750], indices[750:982], indices[982:], indices[928:]

# train_indices, val_indices ,test_indices, test1_indices  = indices[:760], indices[760:1083], indices[1083:], indices[1083:]
test_indices = indices[576:700]+indices[2065:] # indices[2065:]
test1_indices = indices[2065:]
train_val_indices = indices[:576]+indices[700:2065] #indices[:2065]
np.random.seed(random_seed)
np.random.shuffle(train_val_indices)
train_indices = train_val_indices[:1358]
val_indices = train_val_indices[1358:]

# shuffling only train and val indices
np.random.seed(random_seed)
# np.random.shuffle(train_indices)
np.random.seed(random_seed)
# np.random.shuffle(val_indices)

print('train_indices', train_indices)
print('val_indices', val_indices)
print('test_indices', test_indices)
print('test1_indices', test1_indices)

print('train_indices', len(train_indices))
print('val_indices', len(val_indices))
print('test_indices', len(test_indices))
print('test1_indices', len(test1_indices))

## Dataset Class
class Arthopod_Dataset(Dataset):
    def __init__(self, data, transform, indexes):
        self.data = data
        self.transform = transform
        self.indices = indexes
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, index):
        index = self.indices[index]
        # print("********************************* index: ", index)
        session_name = 'session'+str(self.data.loc[index, 'session'])
        img_name = session_name+'-'+str(self.data.loc[index, 'Image'])+'.jpg'
        img_name = os.path.join(DATA_PATH+session_name, img_name)
        # print(img_name)
        image = images_lib[img_name]
        #image = image.convert('RGB')
        # image = image.resize((640,480))
        label = torch.tensor(self.data.loc[index, ['EE_loc', 'O_loc']])
        # if self.transform is not None:
        #     image = self.transform(image)
        return image, label

  

train_dataset = Arthopod_Dataset(data,transform, train_indices)
val_dataset = Arthopod_Dataset(data,transform, val_indices)
test_dataset = Arthopod_Dataset(data,transform, test_indices)
test1_dataset = Arthopod_Dataset(data,transform, test1_indices)

## Creating PT data samplers and loaders:
train_sampler = SequentialSampler( train_indices)
valid_sampler = SequentialSampler( val_indices)
test_sampler = SequentialSampler( test_indices)
test1_sampler = SequentialSampler( test1_indices)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,sampler=test_sampler)
test1_loader = torch.utils.data.DataLoader(test1_dataset, batch_size=batch_size,sampler=test1_sampler)

NameError: ignored

## Neural Net


In [ ]:
class Net(nn.Module):
    # def __init__(self):
    #     super(Net, self).__init__()
    #     # 3 input image channel, 16 output channels, 3x3 square convolution kernel
    #     self.conv1 = nn.Conv2d(3,16,kernel_size=3,stride=2,padding=1)
    #     self.conv2 = nn.Conv2d(16, 32,kernel_size=3,stride=2, padding=1)
    #     self.conv3 = nn.Conv2d(32, 64,kernel_size=3,stride=2, padding=1)
    #     self.conv4 = nn.Conv2d(64, 64,kernel_size=3,stride=2, padding=1)
    #     self.pool = nn.MaxPool2d(2, 2)
    #     self.dropout = nn.Dropout2d(0.4)
    #     self.batchnorm1 = nn.BatchNorm2d(16)
    #     self.batchnorm2 = nn.BatchNorm2d(32)
    #     self.batchnorm3 = nn.BatchNorm2d(64)
    #     self.fc1 = nn.Linear(64*8*10,512 )
    #     self.fc2 = nn.Linear(512, 256)
    #     self.fc3 = nn.Linear(256, 4)

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,9), stride=1)
        self.pool1 = nn.MaxPool2d(4, 3)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,9), stride=1)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=(3,9), stride=1)
        self.pool2 = nn.MaxPool2d(2, 3)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=(3,9), stride=1)
        self.conv5 = nn.Conv2d(32, 32, kernel_size=(3,9), stride=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.fc1_1 = nn.Linear((32*24*24), 128) ### for input shape: [1, 3, 480, 640], the output shape is: [1, 32, 24, 24] and then we concat 44 bounding box
        self.fc1_2 = nn.Linear(128, 64) # Change input dimension
        self.fc1_3 = nn.Linear(64, 32) # Change input dimension
        self.fc1_4 = nn.Linear(32, 4) # Change input  # for ee_loc
        self.fc1_5 = nn.Linear(32, 4) # Change input  # for o_loc
        #self.fc1_6 = nn.Linear(32, 2) # Change input  # for p_label

        
    def forward(self, x):
    #     x = self.batchnorm1(F.relu(self.conv1(x)))
    #     x = self.batchnorm2(F.relu(self.conv2(x)))
    #     x = self.dropout(self.batchnorm2(self.pool(x)))
    #     x = self.batchnorm3(self.pool(F.relu(self.conv3(x))))
    #     x = self.dropout(self.conv4(x))
    #     x = x.view(-1, 64*8*10) # Flatten layer
    #     x = self.dropout(self.fc1(x))
    #     x = self.dropout(self.fc2(x))
    #     x = F.log_softmax(self.fc3(x),dim = 1)
    #     return x
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool2(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.pool3(x) ### for input shape: [1, 3, 480, 640], the output shape is: [1, 32, 24, 24]
        
        # print("Shape of x::::", x.shape)
        x = x.view(-1, 32*24*24) # Flatten layer
        # print("x[0,:]::::::::",x[0,:])
        # print("Shape of x::::", x.shape, "Shape of x_bb::::", x_bb.shape)
        # x = torch.cat((x, x_bb),1)
        # x = x.view()
        x = self.fc1_1(x)
        x = self.fc1_2(x)
        x = self.fc1_3(x)
        x1 = self.fc1_4(x) ## for ee_loc
        x2 = self.fc1_5(x) ## for o_loc
        #x3 = self.fc1_6(x) ## for p_label
        x1 = F.softmax(x1,dim = 1)
        x2 = F.softmax(x2,dim = 1)
        #x3 = F.softmax(x3,dim = 1)
        return x1, x2 #, x3

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

# model = Net() # On CPU
model = Net().to(device)  # On GPU
print(model)


w1 = torch.tensor([443, 525, 211, 762], dtype=torch.float32)
w1 = 1.0 / w1
w1 = w1 / w1.sum()
w1 = torch.FloatTensor(w1).cuda()
criterion1 = nn.CrossEntropyLoss(weight=w1)


w2 = torch.tensor([719, 525, 211, 486], dtype=torch.float32)
w2 = 1.0 / w2
w2 = w2 / w2.sum()
w2 = torch.FloatTensor(w2).cuda()
criterion2= nn.CrossEntropyLoss(weight=w2)

optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()


cuda:0
Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 9), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=4, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 9), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 9), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(32, 32, kernel_size=(3, 9), stride=(1, 1))
  (conv5): Conv2d(32, 32, kernel_size=(3, 9), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1_1): Linear(in_features=18432, out_features=128, bias=True)
  (fc1_2): Linear(in_features=128, out_features=64, bias=True)
  (fc1_3): Linear(in_features=64, out_features=32, bias=True)
  (fc1_4): Linear(in_features=32, out_features=4, bias=True)
  (fc1_5): Linear(in_features=32, out_features=4, bias=True)
)


In [ ]:
w1

tensor([0.17094, 0.37821, 0.34579, 0.10506], device='cuda:0')

In [ ]:
w2

tensor([0.10678, 0.37908, 0.34659, 0.16755], device='cuda:0')

## Training


In [ ]:
n_epochs = 500
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc1 = []
val_acc2 = []
# val_acc3 = []
train_loss = []
train_acc1 = []
train_acc2 = []
# train_acc3 = []
total_step = len(train_loader)

# ## load model state
checkpoint = torch.load("model_classification_tutorial10.pt")
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
# epoch = checkpoint['epoch']
valid_loss_min = checkpoint['loss']


for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    # scheduler.step(epoch)
    correct1 = 0
    correct2 = 0
    # correct3 = 0
    total1=0
    total2=0
    # total3=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_,  target_) in enumerate(train_loader):
        data_, target_ = data_.to(device),  target_.to(device)# on GPU
        # print("target_.shape:::",target_.shape)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs1, outputs2 = model(data_)
        loss1 = criterion1(outputs1, target_[:,0])
        loss2 = criterion2(outputs2, target_[:,1])
        # loss3 = criterion(outputs3, target_[:,2])
        loss = loss1 + loss2 #+ loss3
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        _,pred1 = torch.max(outputs1, dim=1)
        _,pred2 = torch.max(outputs2, dim=1)
        # _,pred3 = torch.max(outputs3, dim=1)
        correct1 += torch.sum(pred1==target_[:,0]).item()
        correct2 += torch.sum(pred2==target_[:,1]).item()
        # correct3 += torch.sum(pred3==target_[:,2]).item()
        total1 += target_[:,0].size(0)
        total2 += target_[:,1].size(0)
        # total3 += target_[:,2].size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc1.append(100 * correct1 / total1)
    train_acc2.append(100 * correct2 / total2)
    # train_acc3.append(100 * correct3 / total3)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss):.4f}, train acc1: {(100 * correct1 / total1):.4f}, train acc2: {(100 * correct2 / total2):.4f}')#', train acc3: {(100 * correct3 / total3):.4f}')
    batch_loss = 0
    total_t1=0
    total_t2=0
    # total_t3=0
    correct_t1=0
    correct_t2=0
    # correct_t3=0
    with torch.no_grad():
        model.eval()
        for data_t,  target_t in (validation_loader):
            data_t,  target_t = data_t.to(device), target_t.to(device)# on GPU
            outputs_t1, outputs_t2 = model(data_t)#, bb_t)
            loss_t1 = criterion1(outputs_t1, target_t[:,0])
            loss_t2 = criterion2(outputs_t2, target_t[:,1])
            # loss_t3 = criterion(outputs_t3, target_t[:,2])
            loss_t = loss_t1 + loss_t2 #+ loss_t3
            batch_loss += loss_t.item()
            _,pred_t1 = torch.max(outputs_t1, dim=1)
            _,pred_t2 = torch.max(outputs_t2, dim=1)
            # _,pred_t3 = torch.max(outputs_t3, dim=1)
            correct_t1 += torch.sum(pred_t1==target_t[:,0]).item()
            correct_t2 += torch.sum(pred_t2==target_t[:,1]).item()
            # correct_t3 += torch.sum(pred_t3==target_t[:,2]).item()
            total_t1 += target_t[:,0].size(0)
            total_t2 += target_t[:,1].size(0)
            # total_t3 += target_t[:,2].size(0)
        # print("Eval::::correct_t1,total_t1", correct_t1, total_t1 )
        val_acc1.append(100 * correct_t1 / total_t1)
        val_acc2.append(100 * correct_t2 / total_t2)
        # val_acc3.append(100 * correct_t3 / total_t3)
        val_loss.append(batch_loss/len(validation_loader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc1: {(100 * correct_t1 / total_t1):.4f}, validation acc2: {(100 * correct_t2 / total_t2):.4f}\n')#, validation acc3: {(100 * correct_t3 / total_t3):.4f}\n')
        # Saving the best weight 
        if network_learned:
            valid_loss_min = batch_loss
            ## Save model
            state = {'epoch': epoch + 1, 'state_dict': model.state_dict(),'optimizer': optimizer.state_dict(), 'loss': valid_loss_min,  }
            torch.save(state, 'model_classification_tutorial11.pt') 
            # state = {'epoch': epoch + 1, 'state_dict': model.state_dict(),
            #  'optimizer': optimizer.state_dict(),  }
            # torch.save(state, 'model_classification_tutorial10.pt')   
            # torch.save(model.state_dict(), 'model_classification_tutorial10.pt')
            print('Detected network improvement, saving current model')
    model.train()

Epoch 1

Epoch [1/500], Step [0/43], Loss: 1.9392
Epoch [1/500], Step [20/43], Loss: 1.6693
Epoch [1/500], Step [40/43], Loss: 2.0384

train loss: 1.8331, train acc1: 84.2415, train acc2: 85.1252
validation loss: 1.7892, validation acc1: 87.8216, validation acc2: 85.9348

Epoch 2

Epoch [2/500], Step [0/43], Loss: 1.7349
Epoch [2/500], Step [20/43], Loss: 1.5152
Epoch [2/500], Step [40/43], Loss: 1.7450

train loss: 1.7459, train acc1: 89.6171, train acc2: 90.5007
validation loss: 1.7203, validation acc1: 89.7084, validation acc2: 89.8799

Epoch 3

Epoch [3/500], Step [0/43], Loss: 1.6020
Epoch [3/500], Step [20/43], Loss: 1.5360
Epoch [3/500], Step [40/43], Loss: 1.7382

train loss: 1.7065, train acc1: 92.4153, train acc2: 91.3108
validation loss: 1.6961, validation acc1: 89.3654, validation acc2: 90.0515

Epoch 4

Epoch [4/500], Step [0/43], Loss: 1.6076
Epoch [4/500], Step [20/43], Loss: 1.5050
Epoch [4/500], Step [40/43], Loss: 1.7319

train loss: 1.6849, train acc1: 92.0471, train

## Testing


In [ ]:
# val_loss = []
test_acc1 = []
test_acc2 = []
# test_acc3 = []
# 
total_t1=0
total_t2=0
# total_t3=0
correct_t1=0
correct_t2=0
# correct_t3=0

allpreds1 = torch.rand(0)
allpreds2 = torch.rand(0)



# allpreds3 = torch.rand(0)

allpreds1 = allpreds1.to(device)
allpreds2 = allpreds2.to(device)
# allpreds3 = allpreds3.to(device)

alltgs1 = torch.rand(0)
alltgs2 = torch.rand(0)
alltgs1 = alltgs1.to(device)
alltgs2 = alltgs2.to(device)

# model.load('model_classification_tutorial.pt')
# model.load_state_dict(torch.load("model_classification_tutorial.pt"))

# ## load model state
# checkpoint = torch.load("model_classification_tutorial10.pt")
# model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer'])
# epoch = checkpoint['epoch']
# valid_loss_min = checkpoint['loss']

# ## load model state
checkpoint = torch.load("model_classification_tutorial16.pt")
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
# epoch = checkpoint['epoch']
valid_loss_min = checkpoint['loss']


with torch.no_grad():
    model.eval()
    for data_t, target_t in (test_loader):
        data_t, target_t = data_t.to(device), target_t.to(device)# on GPU
        outputs_t1, outputs_t2 = model(data_t)
        # loss_t1 = criterion(outputs_t1, target_t[:,0])
        # loss_t2 = criterion(outputs_t2, target_t[:,1])
        # loss_t3 = criterion(outputs_t3, target_t[:,2])
        # loss_t = loss_t1 + loss_t2 + loss_t3
        # batch_loss += loss_t.item()
        _,pred_t1 = torch.max(outputs_t1, dim=1)
        # print(pred_t1.shape)
        _,pred_t2 = torch.max(outputs_t2, dim=1)
        # _,pred_t3 = torch.max(outputs_t3, dim=1)

        alltgs1 = torch.cat((alltgs1,target_t[:,0]),dim=0)
        alltgs2 = torch.cat((alltgs2,target_t[:,1]),dim=0)
        
        allpreds1 = torch.cat((allpreds1,pred_t1),dim=0)
        allpreds2 = torch.cat((allpreds2,pred_t2),dim=0)

        # allpreds3 = torch.cat((allpreds3,pred_t3),dim=0)

        correct_t1 += torch.sum(pred_t1==target_t[:,0]).item()
        correct_t2 += torch.sum(pred_t2==target_t[:,1]).item()
        # correct_t3 += torch.sum(pred_t3==target_t[:,2]).item()
        total_t1 += target_t[:,0].size(0)
        total_t2 += target_t[:,1].size(0)
        # total_t3 += target_t[:,2].size(0)
    # print("Eval::::correct_t1,total_t1", correct_t1, total_t1 )
    test_acc1.append(100 * correct_t1 / total_t1)
    test_acc2.append(100 * correct_t2 / total_t2)
    # test_acc3.append(100 * correct_t3 / total_t3)
    # val_loss.append(batch_loss/len(validation_loader))
    # network_learned = batch_loss < valid_loss_min
    print(f'test acc1: {(100 * correct_t1 / total_t1):.4f}, test acc2: {(100 * correct_t2 / total_t2):.4f}\n') #, test acc3: {(100 * correct_t3 / total_t3):.4f}\n')


test acc1: 82.8010, test acc2: 81.0811



In [ ]:
# torch.save(model.state_dict(), 'model_classification_tutorial5.pt')

## Save model
state = {'epoch': epoch + 1, 'state_dict': model.state_dict(),'optimizer': optimizer.state_dict(), 'loss': valid_loss_min,  }
torch.save(state, 'model_classification_tutorial16.pt') 

In [1]:

out = cv2.VideoWriter('project37.avi',cv2.VideoWriter_fourcc(*'DIVX'), 3, size)


font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (50,100)
fontScale              = 0.5
fontColor              = (0,0,255)
lineType               = 2




ee_loc_vals = {0.0: 'conveyor', 1.0: 'eye', 2.0:'bin', 3.0:'hover'}
o_loc_vals = {0.0: 'conveyor', 1.0: 'eye', 2.0:'bin', 3.0:'hover', 4.0:'placed'}

for i in range(len(img_array)):
  k = i
  tag = 'Frame: '+str(k)+' EE loc: '+ee_loc_vals[eepreds[k]]+' Onion loc: '+o_loc_vals[opreds[k]]


  # img1 = img_array[i].resize((640,480))
  cv2.putText(img1, tag, 
                  bottomLeftCornerOfText, 
                  font, 
                  fontScale,
                  fontColor,
                  lineType)
  
  # img1 = img1.resize((640,480))
  print(i)
  out.write(img_array[i])
out.release()